In [2]:
import pandas as pd
import afqinsight as afq

In [3]:
nodes = pd.read_csv('../data/raw/age_data/nodes.csv')
nodes.head()

,subjectID,tractID,nodeID,rd,fa,cl,ad,md
0,subject_000,Left Thalamic Radiation,0,0.656032,0.183053,0.081921,0.875535,0.729200
1,subject_000,Left Thalamic Radiation,1,0.613308,0.247121,0.115480,0.909085,0.711901
2,subject_000,Left Thalamic Radiation,2,0.574612,0.306726,0.151766,0.944572,0.697932
3,subject_000,Left Thalamic Radiation,3,0.549868,0.343995,0.176124,0.966964,0.688900
4,subject_000,Left Thalamic Radiation,4,0.530190,0.373869,0.194396,0.985039,0.681806


In [4]:
subjects = pd.read_csv('../data/raw/age_data/subjects.csv')
subjects.head()

,Unnamed: 0,subjectID,Age,Gender,Handedness,IQ,IQ_Matrix,IQ_Vocab
0,0,subject_000,20,Male,NaN,139.0,65.0,77.0
1,1,subject_001,31,Male,NaN,129.0,58.0,74.0
2,2,subject_002,18,Female,NaN,130.0,63.0,70.0
3,3,subject_003,28,Male,Right,NaN,NaN,NaN
4,4,subject_004,29,Male,NaN,NaN,NaN,NaN


In [5]:
X, y, groups, feature_names, group_names, subjects, _ = afq.datasets.load_afq_data('../data/raw/age_data', "Age")

In [6]:
X

array([[0.80529484, 0.84255967, 0.89047843, ..., 0.5800756 , 0.61598011,
        0.66923146],
       [0.79958972, 0.83028681, 0.85340832, ..., 0.62736467, 0.64968075,
        0.66583497],
       [0.81828729, 0.8627886 , 0.91714986, ..., 0.6004928 , 0.62281404,
        0.64698932],
       ...,
       [0.79745675, 0.8340532 , 0.85469315, ..., 0.58467594, 0.61122325,
        0.63764813],
       [0.77816811, 0.82947297, 0.87621243, ..., 0.61050187, 0.64257178,
        0.6673916 ],
       [0.82268442, 0.84248315, 0.87065483, ..., 0.61433358, 0.64663074,
        0.68896431]])

In [7]:
y

array([20, 31, 18, 28, 29, 36, 39, 34, 24, 21, 29, 31, 28, 20, 21, 20, 31,
       32, 12, 14, 13, 12, 12, 11,  9, 14,  8, 10,  7,  8, 11, 11, 21,  9,
       24,  8,  8, 14, 10, 14,  7,  8,  9,  8,  8, 11,  6,  9,  8,  9, 32,
       16,  9,  9,  8, 15,  9, 10,  9,  8,  9, 24, 24,  9,  9, 43, 13, 44,
       50, 47, 46, 18, 40, 50, 40, 17, 17])

In [25]:
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from groupyr.transform import GroupExtractor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

In [19]:
group_extractor = GroupExtractor(select=['fa', 'md'], groups=groups, group_names=group_names)
#X = group_extractor.fit_transform(X)

array([[0.21357297, 0.2898142 , 0.34087688, ..., 0.70245914, 0.70967451,
        0.73665818],
       [0.19058299, 0.2721342 , 0.32055065, ..., 0.72820305, 0.73652069,
        0.73947076],
       [0.24233305, 0.30895163, 0.37506668, ..., 0.70167013, 0.70817548,
        0.71672209],
       ...,
       [0.18992911, 0.24986252, 0.27899307, ..., 0.68931342, 0.69524343,
        0.70089865],
       [0.24526717, 0.31377051, 0.36500822, ..., 0.73162777, 0.73706813,
        0.73935197],
       [0.20669057, 0.2846074 , 0.33341608, ..., 0.72049506, 0.73640573,
        0.76009836]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
#X

### Lasso

In [24]:
pipeline = Pipeline([('impute', SimpleImputer(strategy='median')),('estimator', LassoCV(n_alphas=5))])
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

/Users/ceciliabarnes/opt/anaconda3/envs/direct_cnn/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2340466376931083, tolerance: 0.809095652173913
  model = cd_fast.enet_coordinate_descent(


0.15653011665444183

In [11]:
cross_val_score(pipeline, X, y)

/Users/ceciliabarnes/opt/anaconda3/envs/direct_cnn/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8651511819546354, tolerance: 0.865125
  model = cd_fast.enet_coordinate_descent(
/Users/ceciliabarnes/opt/anaconda3/envs/direct_cnn/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9857456835363632, tolerance: 0.8480775510204082
  model = cd_fast.enet_coordinate_descent(
/Users/ceciliabarnes/opt/anaconda3/envs/direct_cnn/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.38953400583074327, tolerance: 0.2079387755102041
  model = cd_fast.enet_coordinate_descent(
/Use

array([-0.32016338,  0.01782182, -2.92530516, -0.5624851 , -0.0597728 ])

### Random Forest

In [26]:
pipeline = Pipeline([('impute', SimpleImputer(strategy='median')),('estimator', RandomForestRegressor(n_estimators=10))])
pipeline.fit(X, y)
pipeline.score(X, y)

0.8850115413449028

In [27]:
cross_val_score(pipeline, X, y)

array([-0.02170353,  0.33872675, -0.28138743, -0.5063403 , -0.13513763])

### Adaboost

In [28]:
pipeline = Pipeline([('impute', SimpleImputer(strategy='median')),('estimator', AdaBoostRegressor())])
pipeline.fit(X, y)
pipeline.score(X, y)

0.98687002029435

In [29]:
cross_val_score(pipeline, X, y)

array([-1.96236055,  0.52854271, -1.26871827, -0.69832751, -0.18904504])